In [1]:
import numpy as np 
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
import matplotlib.pyplot as plt
import string
import re
from collections import Counter
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
#import tensorflow

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kirthika\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kirthika\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('vaderDataset.csv')
df

,Unnamed: 0,Unnamed: 0.1,Tweets,Date,Country,Vader-sentiment value,Vader-sentiment
0,0,0,I wish quick recovery for talented @TOLOnews J...,2020-04-29 23:33:40+00:00,afghanistan,0.9022,positive
1,1,1,These chinese people on campus are so consider...,2020-01-28 23:41:13,afghanistan,0.5777,positive
2,2,2,"China’s Leader, Under Fire, Says He Led Corona...",2020-02-15 23:17:09,afghanistan,-0.6124,negative
3,3,3,RT @DarrenEuronews: Top 5 countries 🌍 with hig...,Sat Jun 13 07:50:54 +0000 2020,afghanistan,0.2732,positive
4,4,4,"RT @MileyCyrus: Spain, you united in solidarit...",Sat Jun 13 09:52:29 +0000 2020,afghanistan,0.6249,positive
...,...,...,...,...,...,...,...
113566,30500,30500,"RT @YourAnonNews: Yesterday, there were 140,92...",Sat Jun 13 11:24:47 +0000 2020,zimbabwe,0.0000,Neutral
113567,30501,30501,"RT @iloharare: In the face of #covid19, we run...",Sat Jun 13 14:12:11 +0000 2020,zimbabwe,-0.3274,negative
113568,30502,30502,RT @SAfridiOfficial: I’ve been feeling unwell ...,Sat Jun 13 09:52:26 +0000 2020,zimbabwe,-0.8020,negative
113569,30503,30503,It must be this Gvt which is worse than corona...,2020-03-02 23:49:25,zimbabwe,-0.4767,negative


In [3]:
df=df.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1)

In [4]:
stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()

In [5]:
df['Tweets']=df['Tweets'].astype('str')

In [6]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [word for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text
df['Text_cleaned'] = df['Tweets'].apply(lambda x: clean_text(x))

In [7]:
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

df['Text_stemmed'] = df['Text_cleaned'].apply(lambda x: stemming(x))

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text
df['Text_lemmatized_stemmed'] = df['Text_stemmed'].apply(lambda x: lemmatizer(x))
df.head()


,Tweets,Date,Country,Vader-sentiment value,Vader-sentiment,Text_cleaned,Text_stemmed,Text_lemmatized_stemmed
0,I wish quick recovery for talented @TOLOnews J...,2020-04-29 23:33:40+00:00,afghanistan,0.9022,positive,"[wish, quick, recovery, talented, tolonews, jo...","[wish, quick, recoveri, talent, tolonew, journ...","[wish, quick, recoveri, talent, tolonew, journ..."
1,These chinese people on campus are so consider...,2020-01-28 23:41:13,afghanistan,0.5777,positive,"[chinese, people, campus, considerate, wear, m...","[chines, peopl, campu, consider, wear, mask, o...","[chine, peopl, campu, consider, wear, mask, ot..."
2,"China’s Leader, Under Fire, Says He Led Corona...",2020-02-15 23:17:09,afghanistan,-0.6124,negative,"[china, leader, fire, says, led, coronavirus, ...","[china, leader, fire, say, led, coronaviru, fi...","[china, leader, fire, say, led, coronaviru, fi..."
3,RT @DarrenEuronews: Top 5 countries 🌍 with hig...,Sat Jun 13 07:50:54 +0000 2020,afghanistan,0.2732,positive,"[rt, darreneuronews, top, countries, highest, ...","[rt, darreneuronew, top, countri, highest, num...","[rt, darreneuronew, top, countri, highest, num..."
4,"RT @MileyCyrus: Spain, you united in solidarit...",Sat Jun 13 09:52:29 +0000 2020,afghanistan,0.6249,positive,"[rt, mileycyrus, spain, united, solidarity, bl...","[rt, mileycyru, spain, unit, solidar, black, l...","[rt, mileycyru, spain, unit, solidar, black, l..."


In [8]:
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder
le = LabelEncoder() 
#df['deceptive']= le.fit_transform(df['deceptive'])
df['polarity']= le.fit_transform(df['Vader-sentiment'])



In [9]:
y = pd.get_dummies(df['polarity']).values
X = df['Text_lemmatized_stemmed']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train

104134    [death, incorrectli, label, coronaviru, death,...
102373    [coronaviru, crisi, made, china, strategist, h...
35694         [coronaviru, love, race, truli, ethnic, viru]
13965                     [stay, safe, everyon, coronaviru]
75874     [rememb, prematur, open, countri, back, week, ...
                                ...                        
56877     [dchd, identifi, possibl, coronaviru, exposur,...
26123     [read, itll, explain, bet, human, havent, clue...
42965     [much, toilet, paper, coronaviru, toilet, pape...
99447     [democraci, leadership, polit, govern, deadlie...
37588     [bill, protect, privat, renter, coronaviru, ou...
Name: Text_lemmatized_stemmed, Length: 90856, dtype: object

In [10]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

TOP_K = 30000

MAX_SEQUENCE_LENGTH = 10000

def sequence_vectorize(train_texts, val_texts):
    tokenizer = text.Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(train_texts)

    x_train = tokenizer.texts_to_sequences(train_texts)
    x_val = tokenizer.texts_to_sequences(val_texts)

    max_length = len(max(x_train, key=len))
    if max_length > MAX_SEQUENCE_LENGTH:
        max_length = MAX_SEQUENCE_LENGTH

    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    x_val = sequence.pad_sequences(x_val, maxlen=max_length)
    return x_train, x_val, tokenizer.word_index, max_length

C:\Users\Kirthika\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Kirthika\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Kirthika\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Kirthika\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWa

In [11]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [12]:
x_train, x_val = X_train.values, X_test.values
x_train, x_val, word_index, max_length = sequence_vectorize(x_train, x_val)
num_features = min(len(word_index) + 1, TOP_K)

In [18]:

import tensorflow as tf
embedding_vector_length = 256


model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(TOP_K, embedding_vector_length, input_length=max_length),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist1 = model.fit(x_train, y_train, validation_data=(x_val, y_test), epochs=20, batch_size=128)

Train on 90856 samples, validate on 22715 samples
Epoch 1/20
90856/90856 [==============================] - 73s 808us/sample - loss: 0.6506 - acc: 0.7333 - val_loss: 0.4335 - val_acc: 0.8384
Epoch 2/20
90856/90856 [==============================] - 75s 824us/sample - loss: 0.4244 - acc: 0.8481 - val_loss: 0.4111 - val_acc: 0.8465
Epoch 3/20
90856/90856 [==============================] - 79s 865us/sample - loss: 0.3630 - acc: 0.8704 - val_loss: 0.4170 - val_acc: 0.8480
Epoch 4/20
90856/90856 [==============================] - 76s 840us/sample - loss: 0.3250 - acc: 0.8849 - val_loss: 0.4267 - val_acc: 0.8506
Epoch 5/20
90856/90856 [==============================] - 76s 842us/sample - loss: 0.2961 - acc: 0.8950 - val_loss: 0.4408 - val_acc: 0.8472
Epoch 6/20
90856/90856 [==============================] - 85s 939us/sample - loss: 0.2769 - acc: 0.9026 - val_loss: 0.4555 - val_acc: 0.8458
Epoch 7/20
90856/90856 [==============================] - 75s 825us/sample - loss: 0.2596 - acc: 0.9094 

In [21]:
accr = model.evaluate(x_val,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

22715/22715 [==============================] - 2s 108us/sample - loss: 0.6238 - acc: 0.8357
Test set
  Loss: 0.624
  Accuracy: 0.836
